In [1]:
import pandas as pd

In [2]:
pip install openpyxl

In [3]:
df1 = pd.read_excel("../input/drug-combination-dataset/15357163mct150843-sup-156849_1_supp_1_w2lrww.xls")

In [4]:
df1.head()

In [5]:
# #specify the columns to sum
# cols = ['viability1','viability2','viability3',"viability4"]

# #find sum of columns specified 
# df1['mean_viability'] = df1[cols].sum(axis=1)

In [6]:
# df1['mean_viability'] = df1['mean_viability']/4 

In [7]:
# df1.head()

In [8]:
drop_list = ["drugA Conc (µM)","drugB Conc (µM)",'viability4']
df = df1.drop(df1.columns.difference(drop_list), axis=1)

In [9]:
df.head()

In [10]:
df.info()

In [11]:
df.isnull().sum()

In [12]:
df = df.dropna()

In [13]:
df.duplicated().sum()

In [14]:
df = df.drop_duplicates()

In [15]:
df.info()

In [16]:
df.describe()

In [17]:
pip install gplearn

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import math
from gplearn.genetic import SymbolicRegressor
import sklearn
from sympy import *
from sympy import symbols, Eq, solve
from sklearn.utils.random import check_random_state
import graphviz
import time
from gplearn.genetic import SymbolicClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")

In [19]:
X = df.iloc[:,0:2]
y = df.iloc[:,2:]

In [20]:
X.head()

In [21]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
scaler = preprocessing.StandardScaler()
standard_X = scaler.fit_transform(X)
standard_X = pd.DataFrame(standard_X, columns =['drugA Conc (µM)','drugB Conc (µM)'])

In [22]:
standard_X.head()

In [23]:
standard_X.describe()

In [24]:
y.head()

In [25]:
y.describe()

In [26]:
converter = {
    'add': lambda x, y : x + y,
    'sub': lambda x, y : x - y,
    'mul': lambda x, y : x*y,
    'div': lambda x, y : x/y,
    'sqrt': lambda x : x**0.5,
    'log': lambda x : log(x),
    'abs': lambda x : abs(x),
    'neg': lambda x : -x,
    'inv': lambda x : 1/x,
    'max': lambda x, y : max(x, y),
    'min': lambda x, y : min(x, y),
    'sin': lambda x : sin(x),
    'cos': lambda x : cos(x),
    'pow': lambda x, y : x**y,
}

In [27]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(standard_X,y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [28]:
# feature=np.array(['drugA Conc (µM)','drugB Conc (µM)'])
function_set = ['add', 'sub', 'mul', 'div','cos','sin','neg','inv']

In [29]:
# model = SymbolicRegressor(population_size=2000,function_set=function_set,
#                            generations=5,parsimony_coefficient=0.00001, random_state=0,
#                           feature_names=X_train.columns)

In [30]:
model = SymbolicRegressor(population_size=5000,generations=50, stopping_criteria=1.79,
                           p_crossover=0.9, p_subtree_mutation=0.0001, p_hoist_mutation=0.0001, p_point_mutation=0.0001,
                           max_samples=0.8, verbose=1,
                           function_set = ('add', 'sub', 'mul', 'div', 
                                           'sqrt', 'log', 'abs', 'neg', 'inv','max', 'min', 
                                           'tan', 'cos', 'sin'    
                                          ),
                           #function_set = (gp_tanh, 'add', 'sub', 'mul', 'div'),
                           metric = 'mean absolute error', warm_start=True,
                           n_jobs = 4, parsimony_coefficient=0.0001, random_state=11)

In [31]:
# from gplearn.genetic import SymbolicTransformer

In [32]:
# model = SymbolicTransformer(population_size=1000, hall_of_fame=100, n_components=10, generations=20, tournament_size=20, stopping_criteria=1.0, const_range=(- 1.0, 1.0), init_depth=(2, 6), init_method='half and half', function_set=function_set, metric='pearson', parsimony_coefficient=0.001, p_crossover=0.9, p_subtree_mutation=0.01, p_hoist_mutation=0.01, p_point_mutation=0.01, p_point_replace=0.05, max_samples=1.0, feature_names=X_train.columns, warm_start=False, low_memory=False, n_jobs=1, verbose=0, random_state=0)

fit

In [33]:
model.fit(X_train, y_train)

predict

In [34]:
y_pred = model.predict(X_test)

In [35]:
model.score(X_test, y_test)

In [36]:
from sklearn.metrics import r2_score

In [41]:
r2 =r2_score(y_test,y_pred,multioutput='variance_weighted')

In [42]:
r2

In [43]:
next_e = sympify(str(model._program), locals=converter)
next_e

In [44]:
from sklearn.metrics import mean_squared_error

In [56]:
mean_squared_error(y_test, y_pred, squared=True)

In [46]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

In [47]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(n_estimators=1000)

In [48]:
reg.fit(X_train,y_train)

In [49]:
predicted_value = reg.predict(X_test)

In [50]:
reg.score(X_test, y_test)

In [57]:
mean_squared_error(y_test, predicted_value, squared=True)

In [58]:
mean_absolute_error(y_test, predicted_value)

In [68]:
plt.rcParams['figure.figsize'] = [8, 8]
plt.scatter(predicted_value, y_test)
plt.xlabel("predicted_value")
plt.ylabel("y_test")
plt.title("Random Forrest Regressor")
plt.show()

In [67]:
plt.rcParams['figure.figsize'] = [8, 8]
plt.scatter(y_pred, y_test)
plt.xlabel("y_pred")
plt.ylabel("y_test")
plt.title("Symbolic Regressor")
plt.show()

In [53]:
pip install synergy

In [54]:
import synergy

In [55]:
from synergy.combination import Bliss 
model = Bliss()
model.fit(df['drugA Conc (µM)'], df['drugB Conc (µM)'], df['viability4'])

In [ ]:
array = model.synergy
array

In [ ]:
min(array)

In [59]:
import statistics

In [60]:
tryw = pd.DataFrame({'list_values':model.synergy})
cleanedlist = list(tryw['list_values'].dropna())
# print("\n* list_var2 = {}".format(cleanedlist))

In [62]:
tryw.describe()

In [ ]:
# # Requires matplotlib
# model.plot_heatmap(xlabel="Drug1", ylabel="Drug2")

# # Requires plotly
# model.plot_surface_plotly(xlabel="Drug1", ylabel="Drug2", 				\
#                           zlabel="Bliss Synergy", fname="plotly.html")

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingRegressor
# define dataset
# X, y = make_regression(n_samples=1000, n_features=20, n_informative=15, noise=0.1, random_state=7)
# define the model
model = GradientBoostingRegressor()
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model
n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
model.fit(X_train,y_train)

In [ ]:
gb = model.predict(X_test)

In [ ]:
print(r2_score(y_test,gb))